In [74]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

In [75]:
stock = pd.read_csv('./dataset/indexData.csv')
stock.head()

,Index,Date,Open,High,Low,Close,Adj Close,Volume
0,NYA,1965-12-31,528.690002,528.690002,528.690002,528.690002,528.690002,0.0
1,NYA,1966-01-03,527.210022,527.210022,527.210022,527.210022,527.210022,0.0
2,NYA,1966-01-04,527.840027,527.840027,527.840027,527.840027,527.840027,0.0
3,NYA,1966-01-05,531.119995,531.119995,531.119995,531.119995,531.119995,0.0
4,NYA,1966-01-06,532.070007,532.070007,532.070007,532.070007,532.070007,0.0


In [76]:
stock.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 112457 entries, 0 to 112456
Data columns (total 8 columns):
 #   Column     Non-Null Count   Dtype  
---  ------     --------------   -----  
 0   Index      112457 non-null  object 
 1   Date       112457 non-null  object 
 2   Open       110253 non-null  float64
 3   High       110253 non-null  float64
 4   Low        110253 non-null  float64
 5   Close      110253 non-null  float64
 6   Adj Close  110253 non-null  float64
 7   Volume     110253 non-null  float64
dtypes: float64(6), object(2)
memory usage: 6.9+ MB


In [77]:
stock[stock.Open.isnull()]

,Index,Date,Open,High,Low,Close,Adj Close,Volume
289,NYA,1967-02-23,NaN,NaN,NaN,NaN,NaN,NaN
26639,HSI,1987-01-01,NaN,NaN,NaN,NaN,NaN,NaN
26659,HSI,1987-01-29,NaN,NaN,NaN,NaN,NaN,NaN
26660,HSI,1987-01-30,NaN,NaN,NaN,NaN,NaN,NaN
26706,HSI,1987-04-06,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
108316,N100,2005-03-28,NaN,NaN,NaN,NaN,NaN,NaN
108511,N100,2005-12-26,NaN,NaN,NaN,NaN,NaN,NaN
110051,N100,2012-01-02,NaN,NaN,NaN,NaN,NaN,NaN
110643,N100,2014-05-01,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
stock.isnull().sum()

Index           0
Date            0
Open         2204
High         2204
Low          2204
Close        2204
Adj Close    2204
Volume       2204
dtype: int64

In [79]:
print(stock.Index.unique())

['NYA' 'IXIC' 'HSI' '000001.SS' 'GSPTSE' '399001.SZ' 'NSEI' 'GDAXI' 'KS11'
 'SSMI' 'TWII' 'J203.JO' 'N225' 'N100']


In [80]:
stock.isnull().sum().max()

2204

In [81]:
stock_dict = {'stock_{}'.format(v.lower()): stock[stock.Index == v].iloc[:, :6] for v in stock.Index.unique()}

In [82]:
for (key,df) in stock_dict.items():
    print('{} total missing values (before): {}'.format(key, df.isnull().sum().max()))
    df.fillna({
        'Open': df.Open.mean(),
        'High': df.High.mean(),
        'Low': df.Low.mean(),
        'Close': df.Close.mean()
    }, inplace=True)   
    print('{} total missing values (after): {}'.format(key, df.isnull().sum().max())) 
    print('\n')

stock_nya total missing values (before): 1
stock_nya total missing values (after): 0



stock_ixic total missing values (before): 0
stock_ixic total missing values (after): 0



stock_hsi total missing values (before): 258
stock_hsi total missing values (after): 0



stock_000001.ss total missing values (before): 172
stock_000001.ss total missing values (after): 0



stock_gsptse total missing values (before): 250
stock_gsptse total missing values (after): 0



stock_399001.sz total missing values (before): 168
stock_399001.sz total missing values (after): 0



stock_nsei total missing values (before): 35
stock_nsei total missing values (after): 0



stock_gdaxi total missing values (before): 168
stock_gdaxi total missing values (after): 0



stock_ks11 total missing values (before): 152
stock_ks11 total missing values (after): 0



stock_ssmi total missing values (before): 159
stock_ssmi total missing values (after): 0



stock_twii total missing values (before): 141
stock_twii total 